In [1]:
import numpy as np
indices = np.random.choice(10, 5, replace=True)
arr = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
print(arr[indices])
print(arr[-11:])
reward_buffer = [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, -1]
size = 7
game_start_index = 0
reward = reward_buffer[size - 1]
updated_rewards = np.empty((size - game_start_index,), int)
updated_rewards[::2] = 1 * reward
updated_rewards[1::2] = -1 * reward
updated_rewards = np.flip(updated_rewards)
print(updated_rewards)
reward_buffer[game_start_index:size] = updated_rewards
print(reward_buffer)
game_start_index = 7
size = 15
print(len(reward_buffer))
reward = reward_buffer[size - 1]
updated_rewards = np.empty((size - game_start_index,), int)
updated_rewards[::2] = 1 * reward
updated_rewards[1::2] = -1 * reward
updated_rewards = np.flip(updated_rewards)
print(updated_rewards)
reward_buffer[game_start_index:size] = updated_rewards
print(reward_buffer)
print(len(reward_buffer))
overflow_reward = 1
overflow_count = 1

[5 3 5 4 9]
[ 1  2  3  4  5  6  7  8  9 10]
[ 1 -1  1 -1  1 -1  1]
[1, -1, 1, -1, 1, -1, 1, 0, 0, 0, 0, 0, 0, 0, -1]
15
[ 1 -1  1 -1  1 -1  1 -1]
[1, -1, 1, -1, 1, -1, 1, 1, -1, 1, -1, 1, -1, 1, -1]
15


In [2]:
# reward_buffer = np.array([1, -1, 1, -1, 1, -1, 0])
# reward_buffer[6] = [1, 1, 1, 1]
# print(reward_buffer)

In [3]:
from alphazero_agent import AlphaZeroAgent
import gymnasium as gym 
import tensorflow as tf

class ClipReward(gym.RewardWrapper):
    def __init__(self, env, min_reward, max_reward):
        super().__init__(env)
        self.min_reward = min_reward
        self.max_reward = max_reward
        self.reward_range = (min_reward, max_reward)

    def reward(self, reward):
        return np.clip(reward, self.min_reward, self.max_reward)


env = ClipReward(gym.wrappers.AtariPreprocessing(gym.make("MsPacmanNoFrameskip-v4", render_mode="rgb_array"), terminal_on_life_loss=True), -1, 1) # as recommended by the original paper, should already include max pooling
env = gym.wrappers.FrameStack(env, 4)

config = {
        'activation': 'relu',
        'kernel_initializer': 'orthogonal',
        'optimizer_function': tf.keras.optimizers.legacy.Adam,
        'learning_rate': 0.2,
        'adam_epsilon': 1e-7,
        'clipnorm': 0.5,
        # NORMALIZATION?
        # REWARD CLIPPING
        'num_epochs': 30,
        'num_filters': 256,
        'kernel_size': 3,
        'stride': 1,
        'num_res_blocks': 5,
        'critic_hidden_filters': 256,
        'critic_dense_size': 256,
        'actor_hidden_filters': 256,
        'actor_dense_size': 256,
        'memory_size': 10000,
        'max_game_length': 1000,
        'batch_size': 32,
        'dirichlet_alpha': 0.3,
        'dirichlet_epsilon': 0.25,
        'c_puct': 4,
        'monte_carlo_simulations': 800,
    }    

agent = AlphaZeroAgent(env, config=config)

/Users/jonathanlamontange-kratz/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


1 Physical GPUs, 1 Logical GPUs


2024-03-04 23:39:13.389828: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-03-04 23:39:13.389854: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-03-04 23:39:13.389861: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-03-04 23:39:13.389897: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-04 23:39:13.389918: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


In [4]:
agent.train()

MCTS Children {0: <MTCS_alphazero.Node object at 0x107ef6160>, 1: <MTCS_alphazero.Node object at 0x116dcdfa0>, 2: <MTCS_alphazero.Node object at 0x116dcd5b0>, 3: <MTCS_alphazero.Node object at 0x116dcdd00>, 4: <MTCS_alphazero.Node object at 0x116dcddf0>, 5: <MTCS_alphazero.Node object at 0x114031370>, 6: <MTCS_alphazero.Node object at 0x11402df40>, 7: <MTCS_alphazero.Node object at 0x11402d4c0>, 8: <MTCS_alphazero.Node object at 0x29dcb5490>}
MCTS Visits [0, 0, 0, 0, 0, 0, 0, 0, 0]
MCTS Max U 0
MCTS Actions [0, 1, 2, 3, 4, 5, 6, 7, 8]
[[0.08612461 0.1169411  0.13657591 0.11273387 0.11281633 0.08252994
  0.11525613 0.10107142 0.13595067]] [[0.16976032]]


IndexError: index 1 is out of bounds for axis 0 with size 1